<a href="https://colab.research.google.com/github/mertege/Thesis_Experiments/blob/main/3_Spectrograms_CI4R_Datasets_Multi_Class_LSTM_with_Attention_Siamese_with_MixUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Siamese-based architecure is running in Gürbüz dataset with all frequency dataset.
# MixUp is added.
# 1 LSTM and GRU blocks are used.
# Bidirectional(GRU) ve Bidirectional(LSTM) for each radar are running in parallel and attention applied seperately. Then, these paths are combined with concatenation. 
# Resolution is 128x170.
# Weights are tied with Siamese method.
# 5 defa run edildi ve değerler aşağıda paylaşıldı.
# Mean test accuracy is 0.932, max test accuracy is 0.941, min test accuracy is 0.916,  std of test accuracy is 0.009.
# Time elapsed through all process: 268.079 sec
# Time elapsed during test time: 4.622 sec

In [2]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate, GRU, LSTM, GRU, TimeDistributed, Bidirectional
from keras.regularizers import l2, l1
import tensorflow as tf
import scipy.io
!pip install numpy==1.19.5
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K 
import gc
import os
!pip install tensorflow-io
import tensorflow_io as tfio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.8 MB 8.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
tensorflow 2.8.2+zzzcolab20220527125636 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.9 MB 2.1 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from keras.layers import *
from keras.models import *
from keras import backend as K

class attention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        # a = K.sigmoid(e)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [5]:
# 77 GHz
bending_77GHz                 = np.load('/content/drive/MyDrive/bending_77Ghz.npy')
label_bending_77GHz           = np.zeros((bending_77GHz.shape[0],1))
label_bending_77GHz[:]        = 1
crawling_77GHz                = np.load('/content/drive/MyDrive/crawling_77Ghz.npy')
label_crawling_77GHz          = np.zeros((crawling_77GHz.shape[0],1))
label_crawling_77GHz[:]       = 2
kneeling_77GHz                = np.load('/content/drive/MyDrive/kneeling_77Ghz.npy')
label_kneeling_77GHz          = np.zeros((kneeling_77GHz.shape[0],1))
label_kneeling_77GHz[:]       = 3
limping_with_RL_Stiff_77GHz   = np.load('/content/drive/MyDrive/limping_with_RL_Stiff_77Ghz.npy')
label_limping_with_RL_Stiff_77GHz           = np.zeros((limping_with_RL_Stiff_77GHz.shape[0],1))
label_limping_with_RL_Stiff_77GHz[:]        = 4
picking_up_an_object_77GHz    = np.load('/content/drive/MyDrive/picking_up_an_object_77Ghz.npy')
label_picking_up_an_object_77GHz          = np.zeros((picking_up_an_object_77GHz.shape[0],1))
label_picking_up_an_object_77GHz[:]       = 5
scissors_gait_77GHz           = np.load('/content/drive/MyDrive/scissors_gait_77Ghz.npy')
label_scissors_gait_77GHz          = np.zeros((scissors_gait_77GHz.shape[0],1))
label_scissors_gait_77GHz[:]       = 6
short_steps_77GHz             = np.load('/content/drive/MyDrive/short_steps_77Ghz.npy')
label_short_steps_77GHz          = np.zeros((short_steps_77GHz.shape[0],1))
label_short_steps_77GHz[:]       = 7
sitting_77GHz                 = np.load('/content/drive/MyDrive/sitting_77Ghz.npy')
label_sitting_77GHz          = np.zeros((sitting_77GHz.shape[0],1))
label_sitting_77GHz[:]       = 8
walking_away_from_Radar_77GHz = np.load('/content/drive/MyDrive/walking_away_from_Radar_77Ghz.npy')
label_walking_away_from_Radar_77GHz         = np.zeros((walking_away_from_Radar_77GHz.shape[0],1))
label_walking_away_from_Radar_77GHz[:]       = 9
Walking_on_both_toes_77GHz    = np.load('/content/drive/MyDrive/Walking_on_both_toes_77Ghz.npy')
label_Walking_on_both_toes_77GHz         = np.zeros((Walking_on_both_toes_77GHz.shape[0],1))
label_Walking_on_both_toes_77GHz[:]       = 10
Walking_towards_radar_77GHz   = np.load('/content/drive/MyDrive/Walking_towards_radar_77Ghz.npy')
label_Walking_towards_radar_77GHz          = np.zeros((Walking_towards_radar_77GHz.shape[0],1))
label_Walking_towards_radar_77GHz[:]       = 11
# 24 GHz
bending_24GHz                 = np.load('/content/drive/MyDrive/bending_24GHz.npy')
label_bending_24GHz           = np.zeros((bending_24GHz.shape[0],1))
label_bending_24GHz[:]        = 1
crawling_24GHz                = np.load('/content/drive/MyDrive/crawling_24GHz.npy')
label_crawling_24GHz          = np.zeros((crawling_24GHz.shape[0],1))
label_crawling_24GHz[:]       = 2
kneeling_24GHz                = np.load('/content/drive/MyDrive/kneeling_24GHz.npy')
label_kneeling_24GHz          = np.zeros((kneeling_24GHz.shape[0],1))
label_kneeling_24GHz[:]       = 3
limping_with_RL_Stiff_24GHz   = np.load('/content/drive/MyDrive/limping with right stiff_24GHz.npy')
label_limping_with_RL_Stiff_24GHz           = np.zeros((limping_with_RL_Stiff_24GHz.shape[0],1))
label_limping_with_RL_Stiff_24GHz[:]        = 4
picking_up_an_object_24GHz    = np.load('/content/drive/MyDrive/picking up object_24GHz.npy')
label_picking_up_an_object_24GHz          = np.zeros((picking_up_an_object_24GHz.shape[0],1))
label_picking_up_an_object_24GHz[:]       = 5
scissors_gait_24GHz           = np.load('/content/drive/MyDrive/scissors gait_24GHz.npy')
label_scissors_gait_24GHz          = np.zeros((scissors_gait_24GHz.shape[0],1))
label_scissors_gait_24GHz[:]       = 6
short_steps_24GHz             = np.load('/content/drive/MyDrive/short steps_24GHz.npy')
label_short_steps_24GHz          = np.zeros((short_steps_24GHz.shape[0],1))
label_short_steps_24GHz[:]       = 7
sitting_24GHz                 = np.load('/content/drive/MyDrive/sitting_24GHz.npy')
label_sitting_24GHz          = np.zeros((sitting_24GHz.shape[0],1))
label_sitting_24GHz[:]       = 8
walking_away_from_Radar_24GHz = np.load('/content/drive/MyDrive/walking away from radar_24GHz.npy')
label_walking_away_from_Radar_24GHz         = np.zeros((walking_away_from_Radar_24GHz.shape[0],1))
label_walking_away_from_Radar_24GHz[:]       = 9
Walking_on_both_toes_24GHz    = np.load('/content/drive/MyDrive/walking on toes both_24GHz.npy')
label_Walking_on_both_toes_24GHz         = np.zeros((Walking_on_both_toes_24GHz.shape[0],1))
label_Walking_on_both_toes_24GHz[:]       = 10
Walking_towards_radar_24GHz   = np.load('/content/drive/MyDrive/walking towards radar_24GHz.npy')
label_Walking_towards_radar_24GHz          = np.zeros((Walking_towards_radar_24GHz.shape[0],1))
label_Walking_towards_radar_24GHz[:]       = 11
# Xethru
bending_Xethru                 = np.load('/content/drive/MyDrive/bending_Xethru.npy')
label_bending_Xethru           = np.zeros((bending_Xethru.shape[0],1))
label_bending_Xethru[:]        = 1
crawling_Xethru                = np.load('/content/drive/MyDrive/crawling_Xethru.npy')
label_crawling_Xethru          = np.zeros((crawling_Xethru.shape[0],1))
label_crawling_Xethru[:]       = 2
kneeling_Xethru                = np.load('/content/drive/MyDrive/kneeling_Xethru.npy')
label_kneeling_Xethru          = np.zeros((kneeling_Xethru.shape[0],1))
label_kneeling_Xethru[:]       = 3
limping_with_RL_Stiff_Xethru   = np.load('/content/drive/MyDrive/limping with right leg stiff_Xethru.npy')
label_limping_with_RL_Stiff_Xethru           = np.zeros((limping_with_RL_Stiff_Xethru.shape[0],1))
label_limping_with_RL_Stiff_Xethru[:]        = 4
picking_up_an_object_Xethru    = np.load('/content/drive/MyDrive/ picking up an object_Xethru.npy')
label_picking_up_an_object_Xethru          = np.zeros((picking_up_an_object_Xethru.shape[0],1))
label_picking_up_an_object_Xethru[:]       = 5
scissors_gait_Xethru           = np.load('/content/drive/MyDrive/scissor Gait_Xethru.npy')
label_scissors_gait_Xethru          = np.zeros((scissors_gait_Xethru.shape[0],1))
label_scissors_gait_Xethru[:]       = 6
short_steps_Xethru             = np.load('/content/drive/MyDrive/short Step_Xethru.npy')
label_short_steps_Xethru          = np.zeros((short_steps_Xethru.shape[0],1))
label_short_steps_Xethru[:]       = 7
sitting_Xethru                 = np.load('/content/drive/MyDrive/sitting_Xethru.npy')
label_sitting_Xethru          = np.zeros((sitting_Xethru.shape[0],1))
label_sitting_Xethru[:]       = 8
walking_away_from_Radar_Xethru = np.load('/content/drive/MyDrive/Walking away from radar_Xethru.npy')
label_walking_away_from_Radar_Xethru         = np.zeros((walking_away_from_Radar_Xethru.shape[0],1))
label_walking_away_from_Radar_Xethru[:]       = 9
Walking_on_both_toes_Xethru    = np.load('/content/drive/MyDrive/walking on both toes_Xethru.npy')
label_Walking_on_both_toes_Xethru         = np.zeros((Walking_on_both_toes_Xethru.shape[0],1))
label_Walking_on_both_toes_Xethru[:]       = 10
Walking_towards_radar_Xethru   = np.load('/content/drive/MyDrive/Walking towards the radar_Xethru.npy')
label_Walking_towards_radar_Xethru          = np.zeros((Walking_towards_radar_Xethru.shape[0],1))
label_Walking_towards_radar_Xethru[:]       = 11

In [6]:
min_length = min(label_bending_77GHz.shape[0],label_bending_24GHz.shape[0],label_bending_Xethru.shape[0])
label_concat = label_bending_77GHz[:min_length]
bending_77GHz = bending_77GHz[:min_length,:,:,:]
bending_24GHz = bending_24GHz[:min_length,:,:,:]
bending_Xethru = bending_Xethru[:min_length,:,:,:]

min_length = min(label_crawling_77GHz.shape[0],label_crawling_24GHz.shape[0],label_crawling_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_crawling_77GHz[:min_length]),axis=0)  
crawling_77GHz = crawling_77GHz[:min_length,:,:,:]
crawling_24GHz = crawling_24GHz[:min_length,:,:,:]
crawling_Xethru = crawling_Xethru[:min_length,:,:,:]

min_length = min(label_kneeling_77GHz.shape[0],label_kneeling_24GHz.shape[0],label_kneeling_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_kneeling_77GHz[:min_length]),axis=0)  
kneeling_77GHz = kneeling_77GHz[:min_length,:,:,:]
kneeling_24GHz = kneeling_24GHz[:min_length,:,:,:]
kneeling_Xethru = kneeling_Xethru[:min_length,:,:,:]

min_length = min(label_limping_with_RL_Stiff_77GHz.shape[0],label_limping_with_RL_Stiff_24GHz.shape[0],label_limping_with_RL_Stiff_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_limping_with_RL_Stiff_77GHz[:min_length]),axis=0)  
limping_with_RL_Stiff_77GHz = limping_with_RL_Stiff_77GHz[:min_length,:,:,:]
limping_with_RL_Stiff_24GHz = limping_with_RL_Stiff_24GHz[:min_length,:,:,:]
limping_with_RL_Stiff_Xethru = limping_with_RL_Stiff_Xethru[:min_length,:,:,:]

min_length = min(label_picking_up_an_object_77GHz.shape[0],label_picking_up_an_object_24GHz.shape[0],label_picking_up_an_object_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_picking_up_an_object_77GHz[:min_length]),axis=0)  
picking_up_an_object_77GHz = picking_up_an_object_77GHz[:min_length,:,:,:]
picking_up_an_object_24GHz = picking_up_an_object_24GHz[:min_length,:,:,:]
picking_up_an_object_Xethru = picking_up_an_object_Xethru[:min_length,:,:,:]

min_length = min(label_scissors_gait_77GHz.shape[0],label_scissors_gait_24GHz.shape[0],label_scissors_gait_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_scissors_gait_77GHz[:min_length]),axis=0)  
scissors_gait_77GHz = scissors_gait_77GHz[:min_length,:,:,:]
scissors_gait_24GHz = scissors_gait_24GHz[:min_length,:,:,:]
scissors_gait_Xethru = scissors_gait_Xethru[:min_length,:,:,:]

min_length = min(label_short_steps_77GHz.shape[0],label_short_steps_24GHz.shape[0],label_short_steps_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_short_steps_77GHz[:min_length]),axis=0)  
short_steps_77GHz = short_steps_77GHz[:min_length,:,:,:]
short_steps_24GHz = short_steps_24GHz[:min_length,:,:,:]
short_steps_Xethru = short_steps_Xethru[:min_length,:,:,:]

min_length = min(label_sitting_77GHz.shape[0],label_sitting_24GHz.shape[0],label_sitting_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_sitting_77GHz[:min_length]),axis=0)  
sitting_77GHz = sitting_77GHz[:min_length,:,:,:]
sitting_24GHz = sitting_24GHz[:min_length,:,:,:]
sitting_Xethru = sitting_Xethru[:min_length,:,:,:]

min_length = min(label_walking_away_from_Radar_77GHz.shape[0],label_walking_away_from_Radar_24GHz.shape[0],label_walking_away_from_Radar_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_walking_away_from_Radar_77GHz[:min_length]),axis=0)  
walking_away_from_Radar_77GHz = walking_away_from_Radar_77GHz[:min_length,:,:,:]
walking_away_from_Radar_24GHz = walking_away_from_Radar_24GHz[:min_length,:,:,:]
walking_away_from_Radar_Xethru = walking_away_from_Radar_Xethru[:min_length,:,:,:]

min_length = min(label_Walking_on_both_toes_77GHz.shape[0],label_Walking_on_both_toes_24GHz.shape[0],label_Walking_on_both_toes_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_Walking_on_both_toes_77GHz[:min_length]),axis=0)  
Walking_on_both_toes_77GHz = Walking_on_both_toes_77GHz[:min_length,:,:,:]
Walking_on_both_toes_24GHz = Walking_on_both_toes_24GHz[:min_length,:,:,:]
Walking_on_both_toes_Xethru = Walking_on_both_toes_Xethru[:min_length,:,:,:]

min_length = min(label_Walking_towards_radar_77GHz.shape[0],label_Walking_towards_radar_24GHz.shape[0],label_Walking_towards_radar_Xethru.shape[0])
label_concat = np.concatenate((label_concat,label_Walking_towards_radar_77GHz[:min_length]),axis=0)  
Walking_towards_radar_77GHz = Walking_towards_radar_77GHz[:min_length,:,:,:]
Walking_towards_radar_24GHz = Walking_towards_radar_24GHz[:min_length,:,:,:]
Walking_towards_radar_Xethru = Walking_towards_radar_Xethru[:min_length,:,:,:]


In [7]:
# 77 GHz
spectrogram_77GHz_concat = np.concatenate((bending_77GHz,crawling_77GHz),axis=0)
del bending_77GHz
del crawling_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,kneeling_77GHz               ),axis=0)
del kneeling_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,limping_with_RL_Stiff_77GHz  ),axis=0)
del limping_with_RL_Stiff_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,picking_up_an_object_77GHz   ),axis=0)
del picking_up_an_object_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,scissors_gait_77GHz          ),axis=0)
del scissors_gait_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,short_steps_77GHz            ),axis=0)
del short_steps_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,sitting_77GHz                ),axis=0)
del sitting_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,walking_away_from_Radar_77GHz),axis=0)
del walking_away_from_Radar_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,Walking_on_both_toes_77GHz   ),axis=0)
del Walking_on_both_toes_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,Walking_towards_radar_77GHz  ),axis=0)
del Walking_towards_radar_77GHz

# 24 GHz
spectrogram_24GHz_concat = np.concatenate((bending_24GHz,crawling_24GHz),axis=0)
del bending_24GHz
del crawling_24GHz
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,kneeling_24GHz               ),axis=0)
del kneeling_24GHz
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,limping_with_RL_Stiff_24GHz  ),axis=0)
del limping_with_RL_Stiff_24GHz
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,picking_up_an_object_24GHz   ),axis=0)
del picking_up_an_object_24GHz
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,scissors_gait_24GHz          ),axis=0)
del scissors_gait_24GHz
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,short_steps_24GHz            ),axis=0)
del short_steps_24GHz
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,sitting_24GHz                ),axis=0)
del sitting_24GHz
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,walking_away_from_Radar_24GHz),axis=0)
del walking_away_from_Radar_24GHz
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,Walking_on_both_toes_24GHz   ),axis=0)
del Walking_on_both_toes_24GHz
spectrogram_24GHz_concat = np.concatenate((spectrogram_24GHz_concat,Walking_towards_radar_24GHz  ),axis=0)
del Walking_towards_radar_24GHz


# Xethru
spectrogram_Xethru_concat = np.concatenate((bending_Xethru,crawling_Xethru),axis=0)
del bending_Xethru
del crawling_Xethru
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,kneeling_Xethru               ),axis=0)
del kneeling_Xethru
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,limping_with_RL_Stiff_Xethru  ),axis=0)
del limping_with_RL_Stiff_Xethru
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,picking_up_an_object_Xethru   ),axis=0)
del picking_up_an_object_Xethru
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,scissors_gait_Xethru          ),axis=0)
del scissors_gait_Xethru
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,short_steps_Xethru            ),axis=0)
del short_steps_Xethru
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,sitting_Xethru                ),axis=0)
del sitting_Xethru
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,walking_away_from_Radar_Xethru),axis=0)
del walking_away_from_Radar_Xethru
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,Walking_on_both_toes_Xethru   ),axis=0)
del Walking_on_both_toes_Xethru
spectrogram_Xethru_concat = np.concatenate((spectrogram_Xethru_concat,Walking_towards_radar_Xethru  ),axis=0)
del Walking_towards_radar_Xethru


In [8]:
# One-Hot Encode Label
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label_concat)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
print(onehot_encoded_labels)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [10]:
# Concat range-doppler data
range_doppler_concat = spectrogram_77GHz_concat
range_doppler_concat_label = label_concat
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]
# Concat range-doppler data
spectrogram_concat = spectrogram_24GHz_concat
spectrogram_concat_label = onehot_encoded_labels
# Shuffle concat range doppler
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]
# Third Spectrogram
third_spectrogram_concat = spectrogram_Xethru_concat
third_spectrogram_concat_shuffle = third_spectrogram_concat[shuffle_indx,:,:,:]
del range_doppler_concat
del spectrogram_77GHz_concat
del spectrogram_24GHz_concat
del spectrogram_concat
del spectrogram_Xethru_concat
del third_spectrogram_concat

In [11]:
range_doppler_concat_shuffle_new = np.zeros((630,128,170,1))
for ii in range(range_doppler_concat_shuffle.shape[0]):
  aa = range_doppler_concat_shuffle[ii,:,:,0]
  range_doppler_concat_shuffle_new[ii,:,:,0] = cv2.resize(aa, (range_doppler_concat_shuffle_new.shape[2], range_doppler_concat_shuffle_new.shape[1]),interpolation = cv2.INTER_CUBIC)
range_doppler_concat_shuffle = range_doppler_concat_shuffle_new
del aa
del range_doppler_concat_shuffle_new

spectrogram_concat_shuffle_new = np.zeros((630,128,170,1))
for ii in range(spectrogram_concat_shuffle.shape[0]):
  aa = spectrogram_concat_shuffle[ii,:,:,0]
  spectrogram_concat_shuffle_new[ii,:,:,0] = cv2.resize(aa, (spectrogram_concat_shuffle_new.shape[2], spectrogram_concat_shuffle_new.shape[1]),interpolation = cv2.INTER_CUBIC)
spectrogram_concat_shuffle = spectrogram_concat_shuffle_new
del aa
del spectrogram_concat_shuffle_new

third_spectrogram_concat_shuffle_new = np.zeros((630,128,170,1))
for ii in range(third_spectrogram_concat_shuffle.shape[0]):
  aa = third_spectrogram_concat_shuffle[ii,:,:,0]
  third_spectrogram_concat_shuffle_new[ii,:,:,0] = cv2.resize(aa, (third_spectrogram_concat_shuffle_new.shape[2], third_spectrogram_concat_shuffle_new.shape[1]),interpolation = cv2.INTER_CUBIC)
third_spectrogram_concat_shuffle = third_spectrogram_concat_shuffle_new
del aa
del third_spectrogram_concat_shuffle_new

In [12]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
def mask_augmentation(images,range_doppler_training_data,third_spectrogram_training_data, labels, repeat_of_mixup, time_interval):
  concat_images = images
  concat_images_range_doppler = range_doppler_training_data
  concat_images_third_spectrogram = third_spectrogram_training_data
  concat_label = labels  
  if repeat_of_mixup == 0:
    concat_images = images
    concat_images_range_doppler = range_doppler_training_data
    concat_images_third_spectrogram = third_spectrogram_training_data
    concat_label = labels   
  else:  
    images_time_mask = np.zeros((images.shape))
    range_doppler_training_data_time_mask = np.zeros((range_doppler_training_data.shape))
    third_spectrogram_training_data_time_mask = np.zeros((third_spectrogram_training_data.shape))

    for ii in range(repeat_of_mixup):
      for jj in range(images.shape[0]):
        images_time_mask[jj,:,:,0] = tfio.audio.time_mask(images[jj,:,:,0], param = time_interval)
        images_time_mask[jj,:,:,0] = tfio.audio.time_mask(images_time_mask[jj,:,:,0], param = time_interval)
        images_time_mask[jj,:,:,0] = tfio.audio.freq_mask(images_time_mask[jj,:,:,0], param = time_interval)
        images_time_mask[jj,:,:,0] = tfio.audio.freq_mask(images_time_mask[jj,:,:,0], param = time_interval)
        range_doppler_training_data_time_mask[jj,:,:,0] = tfio.audio.time_mask(range_doppler_training_data[jj,:,:,0], param = time_interval)
        range_doppler_training_data_time_mask[jj,:,:,0] = tfio.audio.time_mask(range_doppler_training_data_time_mask[jj,:,:,0], param = time_interval)
        range_doppler_training_data_time_mask[jj,:,:,0] = tfio.audio.freq_mask(range_doppler_training_data_time_mask[jj,:,:,0], param = time_interval)
        range_doppler_training_data_time_mask[jj,:,:,0] = tfio.audio.freq_mask(range_doppler_training_data_time_mask[jj,:,:,0], param = time_interval)
        third_spectrogram_training_data_time_mask[jj,:,:,0] = tfio.audio.time_mask(third_spectrogram_training_data[jj,:,:,0], param = time_interval)
        third_spectrogram_training_data_time_mask[jj,:,:,0] = tfio.audio.time_mask(third_spectrogram_training_data_time_mask[jj,:,:,0], param = time_interval)
        third_spectrogram_training_data_time_mask[jj,:,:,0] = tfio.audio.freq_mask(third_spectrogram_training_data_time_mask[jj,:,:,0], param = time_interval)
        third_spectrogram_training_data_time_mask[jj,:,:,0] = tfio.audio.freq_mask(third_spectrogram_training_data_time_mask[jj,:,:,0], param = time_interval)

      concat_images = np.concatenate((concat_images, images_time_mask))
      concat_images_range_doppler = np.concatenate((concat_images_range_doppler, range_doppler_training_data_time_mask))
      concat_images_third_spectrogram = np.concatenate((concat_images_third_spectrogram, third_spectrogram_training_data))
      concat_label = np.concatenate((concat_label, labels))

  return (concat_images,concat_images_range_doppler,concat_images_third_spectrogram, concat_label)

def mixup_augmentation(images,range_doppler_training_data,third_spectrogram_training_data, labels, repeat_of_mixup, alpha, beta):
    batch_size = images.shape[0]
    concat_images = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_images_third_spectrogram = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))

    images_mixup_range_doppler = range_doppler_training_data
    images_mixup_images = images
    images_mixup_third_spectrogram_training_data = third_spectrogram_training_data
    labels_int = (np.argmax(labels, axis=1))
    labels_int =  labels_int[:, np.newaxis]
    concat_label = labels_int
    if repeat_of_mixup == 0:
      concat_images = images
      concat_images_range_doppler = range_doppler_training_data
      concat_images_third_spectrogram = third_spectrogram_training_data
      concat_label = labels      
    else:
      for ii in range(repeat_of_mixup):
# # --------------------------------------------in-class mixup----------------------------------------------    
        int_indexes = (np.argmax(labels, axis=1))
        gaussian_mean = 0.15
        gaussian_std = 0.05
        

        indx_0 = np.where(int_indexes == 0)  
        indx_0 = np.asarray(indx_0)
        indx_0_size = (indx_0.shape[1])
        ll_0 = np.random.normal(gaussian_mean, gaussian_std, (indx_0_size,1,1,1))
        x_l_0 = np.reshape(ll_0, (indx_0_size,1,1,1))
        y_l_0 = np.reshape(ll_0, (indx_0_size,1))
        shuffle_indx_0 = random.sample(list(indx_0[0]), len(indx_0[0]))
        shuffle_indx_0_2 = random.sample(list(indx_0[0]), len(indx_0[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_0,:,:,:] * x_l_0 + range_doppler_training_data[shuffle_indx_0_2,:,:,:] * (1 - x_l_0)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_0,:,:,:] * x_l_0 + images[shuffle_indx_0_2,:,:,:] * (1 - x_l_0)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_0,:,:,:] * x_l_0 + third_spectrogram_training_data[shuffle_indx_0_2,:,:,:] * (1 - x_l_0)))
       
        integer_encoded = np.zeros((indx_0_size,1))
        integer_encoded[:,0] = 0
        concat_label = np.concatenate((concat_label, integer_encoded))

        indx_1 = np.where(int_indexes == 1)  
        indx_1 = np.asarray(indx_1)
        indx_1_size = (indx_1.shape[1])
        ll_1 = np.random.normal(gaussian_mean, gaussian_std, (indx_1_size,1,1,1))
        x_l_1 = np.reshape(ll_1, (indx_1_size,1,1,1))
        y_l_1 = np.reshape(ll_1, (indx_1_size,1))
        shuffle_indx_1 = random.sample(list(indx_1[0]), len(indx_1[0]))
        shuffle_indx_1_2 = random.sample(list(indx_1[0]), len(indx_1[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_1,:,:,:] * x_l_1 + range_doppler_training_data[shuffle_indx_1_2,:,:,:] * (1 - x_l_1)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_1,:,:,:] * x_l_1 + images[shuffle_indx_1_2,:,:,:] * (1 - x_l_1)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_1,:,:,:] * x_l_1 + third_spectrogram_training_data[shuffle_indx_1_2,:,:,:] * (1 - x_l_1)))

        integer_encoded = np.zeros((indx_1_size,1))
        integer_encoded[:,0] = 1
        concat_label = np.concatenate((concat_label, integer_encoded))
        indx_2 = np.where(int_indexes == 2)   
        indx_2 = np.asarray(indx_2)
        indx_2_size = (indx_2.shape[1])
        ll_2 = np.random.normal(gaussian_mean, gaussian_std, (indx_2_size,1,1,1))
        x_l_2 = np.reshape(ll_2, (indx_2_size,1,1,1))
        y_l_2 = np.reshape(ll_2, (indx_2_size,1))
        shuffle_indx_2 = random.sample(list(indx_2[0]), len(indx_2[0]))
        shuffle_indx_2_2 = random.sample(list(indx_2[0]), len(indx_2[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_2,:,:,:] * x_l_2 + range_doppler_training_data[shuffle_indx_2_2,:,:,:] * (1 - x_l_2)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_2,:,:,:] * x_l_2 + images[shuffle_indx_2_2,:,:,:] * (1 - x_l_2)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_2,:,:,:] * x_l_2 + third_spectrogram_training_data[shuffle_indx_2_2,:,:,:] * (1 - x_l_2)))

        integer_encoded = np.zeros((indx_2_size,1))
        integer_encoded[:,0] = 2
        concat_label = np.concatenate((concat_label, integer_encoded))
        indx_3 = np.where(int_indexes == 3)  
        indx_3 = np.asarray(indx_3)
        indx_3_size = (indx_3.shape[1])
        ll_3 = np.random.normal(gaussian_mean, gaussian_std, (indx_3_size,1,1,1))
        x_l_3 = np.reshape(ll_3, (indx_3_size,1,1,1))
        y_l_3= np.reshape(ll_3, (indx_3_size,1))
        shuffle_indx_3 = random.sample(list(indx_3[0]), len(indx_3[0]))
        shuffle_indx_3_2 = random.sample(list(indx_3[0]), len(indx_3[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_3,:,:,:] * x_l_3 + range_doppler_training_data[shuffle_indx_3_2,:,:,:] * (1 - x_l_3)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_3,:,:,:] * x_l_3 + images[shuffle_indx_3_2,:,:,:] * (1 - x_l_3)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_3,:,:,:] * x_l_3 + third_spectrogram_training_data[shuffle_indx_3_2,:,:,:] * (1 - x_l_3)))

        integer_encoded = np.zeros((indx_3_size,1))
        integer_encoded[:,0] = 3
        concat_label = np.concatenate((concat_label, integer_encoded))
        indx_4 = np.where(int_indexes == 4)  
        indx_4 = np.asarray(indx_4)
        indx_4_size = (indx_4.shape[1])
        ll_4 = np.random.normal(gaussian_mean, gaussian_std, (indx_4_size,1,1,1))
        x_l_4 = np.reshape(ll_4, (indx_4_size,1,1,1))
        y_l_4= np.reshape(ll_4, (indx_4_size,1))
        shuffle_indx_4 = random.sample(list(indx_4[0]), len(indx_4[0]))
        shuffle_indx_4_2 = random.sample(list(indx_4[0]), len(indx_4[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_4,:,:,:] * x_l_4 + range_doppler_training_data[shuffle_indx_4_2,:,:,:] * (1 - x_l_4)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_4,:,:,:] * x_l_4 + images[shuffle_indx_4_2,:,:,:] * (1 - x_l_4)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_4,:,:,:] * x_l_4 + third_spectrogram_training_data[shuffle_indx_4_2,:,:,:] * (1 - x_l_4)))

        integer_encoded = np.zeros((indx_4_size,1))
        integer_encoded[:,0] = 4
        concat_label = np.concatenate((concat_label, integer_encoded))
        indx_5 = np.where(int_indexes == 5)  
        indx_5 = np.asarray(indx_5)
        indx_5_size = (indx_5.shape[1])
        ll_5 = np.random.normal(gaussian_mean, gaussian_std, (indx_5_size,1,1,1))
        x_l_5 = np.reshape(ll_5, (indx_5_size,1,1,1))
        y_l_5= np.reshape(ll_5, (indx_5_size,1))
        shuffle_indx_5 = random.sample(list(indx_5[0]), len(indx_5[0]))
        shuffle_indx_5_2 = random.sample(list(indx_5[0]), len(indx_5[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_5,:,:,:] * x_l_5 + range_doppler_training_data[shuffle_indx_5_2,:,:,:] * (1 - x_l_5)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_5,:,:,:] * x_l_5 + images[shuffle_indx_5_2,:,:,:] * (1 - x_l_5)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_5,:,:,:] * x_l_5 + third_spectrogram_training_data[shuffle_indx_5_2,:,:,:] * (1 - x_l_5)))

        integer_encoded = np.zeros((indx_5_size,1))
        integer_encoded[:,0] = 5
        concat_label = np.concatenate((concat_label, integer_encoded))
        indx_6 = np.where(int_indexes == 6)  
        indx_6 = np.asarray(indx_6)
        indx_6_size = (indx_6.shape[1])
        ll_6 = np.random.normal(gaussian_mean, gaussian_std, (indx_6_size,1,1,1))
        x_l_6 = np.reshape(ll_6, (indx_6_size,1,1,1))
        y_l_6= np.reshape(ll_6, (indx_6_size,1))
        shuffle_indx_6 = random.sample(list(indx_6[0]), len(indx_6[0]))
        shuffle_indx_6_2 = random.sample(list(indx_6[0]), len(indx_6[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_6,:,:,:] * x_l_6 + range_doppler_training_data[shuffle_indx_6_2,:,:,:] * (1 - x_l_6)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_6,:,:,:] * x_l_6 + images[shuffle_indx_6_2,:,:,:] * (1 - x_l_6)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_6,:,:,:] * x_l_6 + third_spectrogram_training_data[shuffle_indx_6_2,:,:,:] * (1 - x_l_6)))

        integer_encoded = np.zeros((indx_6_size,1))
        integer_encoded[:,0] = 6
        concat_label = np.concatenate((concat_label, integer_encoded))
        indx_7 = np.where(int_indexes == 7)  
        indx_7 = np.asarray(indx_7)
        indx_7_size = (indx_7.shape[1])
        ll_7 = np.random.normal(gaussian_mean, gaussian_std, (indx_7_size,1,1,1))
        x_l_7 = np.reshape(ll_7, (indx_7_size,1,1,1))
        y_l_7= np.reshape(ll_7, (indx_7_size,1))
        shuffle_indx_7 = random.sample(list(indx_7[0]), len(indx_7[0]))
        shuffle_indx_7_2 = random.sample(list(indx_7[0]), len(indx_7[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_7,:,:,:] * x_l_7 + range_doppler_training_data[shuffle_indx_7_2,:,:,:] * (1 - x_l_7)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_7,:,:,:] * x_l_7 + images[shuffle_indx_7_2,:,:,:] * (1 - x_l_7)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_7,:,:,:] * x_l_7 + third_spectrogram_training_data[shuffle_indx_7_2,:,:,:] * (1 - x_l_7)))

        integer_encoded = np.zeros((indx_7_size,1))
        integer_encoded[:,0] = 7
        concat_label = np.concatenate((concat_label, integer_encoded))
        indx_8 = np.where(int_indexes == 8)   
        indx_8 = np.asarray(indx_8)
        indx_8_size = (indx_8.shape[1])
        ll_8 = np.random.normal(gaussian_mean, gaussian_std, (indx_8_size,1,1,1))
        x_l_8 = np.reshape(ll_8, (indx_8_size,1,1,1))
        y_l_8= np.reshape(ll_8, (indx_8_size,1))
        shuffle_indx_8 = random.sample(list(indx_8[0]), len(indx_8[0]))
        shuffle_indx_8_2 = random.sample(list(indx_8[0]), len(indx_8[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_8,:,:,:] * x_l_8 + range_doppler_training_data[shuffle_indx_8_2,:,:,:] * (1 - x_l_8)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_8,:,:,:] * x_l_8 + images[shuffle_indx_8_2,:,:,:] * (1 - x_l_8)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_8,:,:,:] * x_l_8 + third_spectrogram_training_data[shuffle_indx_8_2,:,:,:] * (1 - x_l_8)))

        integer_encoded = np.zeros((indx_8_size,1))
        integer_encoded[:,0] = 8
        concat_label = np.concatenate((concat_label, integer_encoded))
        indx_9 = np.where(int_indexes == 9)  
        indx_9 = np.asarray(indx_9)
        indx_9_size = (indx_9.shape[1])
        ll_9 = np.random.normal(gaussian_mean, gaussian_std, (indx_9_size,1,1,1))
        x_l_9 = np.reshape(ll_9, (indx_9_size,1,1,1))
        y_l_9= np.reshape(ll_9, (indx_9_size,1))
        shuffle_indx_9 = random.sample(list(indx_9[0]), len(indx_9[0]))
        shuffle_indx_9_2 = random.sample(list(indx_9[0]), len(indx_9[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_9,:,:,:] * x_l_9 + range_doppler_training_data[shuffle_indx_9_2,:,:,:] * (1 - x_l_9)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_9,:,:,:] * x_l_9 + images[shuffle_indx_9_2,:,:,:] * (1 - x_l_9)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_9,:,:,:] * x_l_9 + third_spectrogram_training_data[shuffle_indx_9_2,:,:,:] * (1 - x_l_9)))

        integer_encoded = np.zeros((indx_9_size,1))
        integer_encoded[:,0] = 9
        concat_label = np.concatenate((concat_label, integer_encoded))
        indx_10 = np.where(int_indexes == 10)  
        indx_10 = np.asarray(indx_10)
        indx_10_size = (indx_10.shape[1])
        ll_10 = np.random.normal(gaussian_mean, gaussian_std, (indx_10_size,1,1,1))
        x_l_10 = np.reshape(ll_10, (indx_10_size,1,1,1))
        y_l_10= np.reshape(ll_10, (indx_10_size,1))
        shuffle_indx_10 = random.sample(list(indx_10[0]), len(indx_10[0]))
        shuffle_indx_10_2 = random.sample(list(indx_10[0]), len(indx_10[0]))
        images_mixup_range_doppler = np.concatenate((images_mixup_range_doppler, range_doppler_training_data[shuffle_indx_10,:,:,:] * x_l_10 + range_doppler_training_data[shuffle_indx_10_2,:,:,:] * (1 - x_l_10)))
        images_mixup_images = np.concatenate((images_mixup_images, images[shuffle_indx_10,:,:,:] * x_l_10 + images[shuffle_indx_10_2,:,:,:] * (1 - x_l_10)))
        images_mixup_third_spectrogram_training_data = np.concatenate((images_mixup_third_spectrogram_training_data, third_spectrogram_training_data[shuffle_indx_10,:,:,:] * x_l_10 + third_spectrogram_training_data[shuffle_indx_10_2,:,:,:] * (1 - x_l_10)))

        integer_encoded = np.zeros((indx_10_size,1))
        integer_encoded[:,0] = 10
        concat_label = np.concatenate((concat_label, integer_encoded))

      for jj in range(images.shape[0]):
        images_mixup_images[jj,:,:,0] = tfio.audio.time_mask(images_mixup_images[jj,:,:,0], param = 10)
        images_mixup_range_doppler[jj,:,:,0] = tfio.audio.time_mask(images_mixup_range_doppler[jj,:,:,0], param = 10)      
        images_mixup_third_spectrogram_training_data[jj,:,:,0] = tfio.audio.time_mask(images_mixup_third_spectrogram_training_data[jj,:,:,0], param = 10)
        images_mixup_images[jj,:,:,0] = tfio.audio.time_mask(images_mixup_images[jj,:,:,0], param = 10)
        images_mixup_range_doppler[jj,:,:,0] = tfio.audio.time_mask(images_mixup_range_doppler[jj,:,:,0], param = 10)
        images_mixup_third_spectrogram_training_data[jj,:,:,0] = tfio.audio.time_mask(images_mixup_third_spectrogram_training_data[jj,:,:,0], param = 10)

        # images_mixup_images[jj,:,:,0] = tfio.audio.freq_mask(images_mixup_images[jj,:,:,0], param = 10)
        # images_mixup_range_doppler[jj,:,:,0] = tfio.audio.freq_mask(images_mixup_range_doppler[jj,:,:,0], param = 10)  
        # images_mixup_third_spectrogram_training_data[jj,:,:,0] = tfio.audio.freq_mask(images_mixup_third_spectrogram_training_data[jj,:,:,0], param = 10)
        # images_mixup_images[jj,:,:,0] = tfio.audio.freq_mask(images_mixup_images[jj,:,:,0], param = 10)
        # images_mixup_range_doppler[jj,:,:,0] = tfio.audio.freq_mask(images_mixup_range_doppler[jj,:,:,0], param = 10)  
        # images_mixup_third_spectrogram_training_data[jj,:,:,0] = tfio.audio.freq_mask(images_mixup_third_spectrogram_training_data[jj,:,:,0], param = 10)

      concat_label = onehot_encoder.fit_transform(concat_label)
      concat_images = images_mixup_images
      concat_images_range_doppler = images_mixup_range_doppler
      concat_images_third_spectrogram = images_mixup_third_spectrogram_training_data
    return (concat_images,concat_images_range_doppler,concat_images_third_spectrogram, concat_label)

def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,third_spectrogram,range_doppler_concat_label_shuffle_train,\
                                       repeat_of_mixup, augmentation_enable):
  # ---------------- Parameters ----------------
  # to split train and validation with same distribution
  alpha = 0.1
  beta = 5
  dummy_label = np.zeros((spectrogram_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(spectrogram_concat_shuffle_train,dummy_label):   
    randomlist_for_validation_indx
  # size_of_validation = int(0.2*spectrogram_concat_shuffle_train.shape[0])
  # # Split validation
  # randomlist_for_validation_indx = random.sample(range(0, range_doppler_concat_shuffle_train.shape[0]), size_of_validation) # split validation data
  # randomlist_for_train_indx = np.delete(range(0, range_doppler_concat_shuffle_train.shape[0]), randomlist_for_validation_indx) # split training data
  # get validation data
  third_spectrogram_validation_data = third_spectrogram[randomlist_for_validation_indx,:,:,:]
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  spectrogram_validation_labels = range_doppler_concat_label_shuffle_train[randomlist_for_validation_indx,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  # get training data
  third_spectrogram_training_data = third_spectrogram[randomlist_for_train_indx,:,:,:]
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  spectrogram_training_labels = spectrogram_concat_label_shuffle_train[randomlist_for_train_indx,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]

  (spectrogram_augmented_image,range_doppler_augmented_image,third_spectrogram_augmented_image,spectrograms_label)=\
   mixup_augmentation(spectrogram_training_data,range_doppler_training_data,third_spectrogram_training_data, spectrogram_training_labels, repeat_of_mixup, alpha, beta)

  time_interval = 10
  # (spectrogram_augmented_image,range_doppler_augmented_image,third_spectrogram_augmented_image,spectrograms_label)=\
  #  mask_augmentation(spectrogram_training_data,range_doppler_training_data,third_spectrogram_training_data, spectrogram_training_labels, repeat_of_mixup, time_interval)
   
   

  return (spectrogram_augmented_image,range_doppler_augmented_image,third_spectrogram_augmented_image,spectrograms_label,\
     spectrogram_validation_data,range_doppler_validation_data,third_spectrogram_validation_data, spectrogram_validation_labels)



def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


In [13]:
normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
third_spectrogram_concat_shuffle = normalize_inputs(third_spectrogram_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)
third_spectrogram_concat_shuffle = np.float32(third_spectrogram_concat_shuffle)


In [14]:
n_features = range_doppler_concat_shuffle.shape[1]
n_steps = range_doppler_concat_shuffle.shape[2]
range_doppler_concat_shuffle = np.transpose(range_doppler_concat_shuffle, axes = (0,2,1,3)) 
spectrogram_concat_shuffle = np.transpose(spectrogram_concat_shuffle, axes = (0,2,1,3)) 
third_spectrogram_concat_shuffle = np.transpose(third_spectrogram_concat_shuffle, axes = (0,2,1,3)) 

In [15]:

# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 128
dropout_prob_dense = 0.3
repeat_of_mixup = 2
number_of_repeat = 5
lstm_dropout_rate = 0.25
unit_number_of_lstm = 64
recurrent_dropout_rate = 0.0
recurrent_regularizer_value = l1(0.0000)
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  test_accuracy_per_fold_77GHz = []
  f1_score_per_fold = []
  for train, test in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):   
    gc.collect()
    K.clear_session()

    randomlist_for_test_indx = test
    randomlist_for_train_indx = train
    # test data
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_test_indx,:]
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    third_spectrogram_concat_shuffle_test = third_spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    #train data
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    third_spectrogram_concat_shuffle_train = third_spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    # ---------------- Split labels to equal them during augmentation for Validation ----------------

            
    (spectrogram_augmented_image,range_doppler_augmented_image,third_spectrogram_augmented_image,spectrogram_concat_label_shuffle_concat,\
     validation_spectrogram,validation_range_doppler,validation_third_spectrogram, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,third_spectrogram_concat_shuffle_train,\
                                         spectrogram_concat_label_shuffle_train,\
                                         repeat_of_mixup, augmentation_enable)
   
    def lstm_encoder_network_1(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = lstm_dropout_rate,
                              recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(input)                            
        x = AveragePooling1D(pool_size=2, strides=None)(x)
        # x = Bidirectional(LSTM(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x)      
        # x = Bidirectional(LSTM(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x) 
        x = tf.transpose(x, perm=[0,2,1])
        x = attention(return_sequences=True)(x)
        x = tf.transpose(x, perm=[0,2,1])        
        x = GlobalAveragePooling1D()(x)

        return Model(input, x)

    def lstm_encoder_network_2(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = lstm_dropout_rate,
                              recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(input)                            
        x = AveragePooling1D(pool_size=2, strides=None)(x)
        # x = Bidirectional(LSTM(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x)      
        # x = Bidirectional(LSTM(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x) 
        x = tf.transpose(x, perm=[0,2,1])
        x = attention(return_sequences=True)(x)
        x = tf.transpose(x, perm=[0,2,1])        
        x = GlobalAveragePooling1D()(x)

        return Model(input, x)

    def lstm_encoder_network_3(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = lstm_dropout_rate,
                              recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(input)                            
        x = AveragePooling1D(pool_size=2, strides=None)(x)
        # x = Bidirectional(LSTM(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x)      
        # x = Bidirectional(LSTM(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x) 
        x = tf.transpose(x, perm=[0,2,1])
        x = attention(return_sequences=True)(x)
        x = tf.transpose(x, perm=[0,2,1])        
        x = GlobalAveragePooling1D()(x)

        return Model(input, x)

    def lstm_encoder_network_1_gru(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(GRU(unit_number_of_lstm, return_sequences=True, dropout = lstm_dropout_rate,
                              recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(input)                    
        x = AveragePooling1D(pool_size=2, strides=None)(x)
        # x = Bidirectional(GRU(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x)      
        # x = Bidirectional(GRU(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x) 
        x = tf.transpose(x, perm=[0,2,1])
        x = attention(return_sequences=True)(x)
        x = tf.transpose(x, perm=[0,2,1])        
        x = GlobalAveragePooling1D()(x)

        return Model(input, x)

    def lstm_encoder_network_2_gru(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(GRU(unit_number_of_lstm, return_sequences=True, dropout = lstm_dropout_rate,
                              recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(input)                    
        x = AveragePooling1D(pool_size=2, strides=None)(x)
        # x = Bidirectional(GRU(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x)      
        # x = Bidirectional(GRU(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x)                       
        x = tf.transpose(x, perm=[0,2,1])
        x = attention(return_sequences=True)(x)
        x = tf.transpose(x, perm=[0,2,1])        
        x = GlobalAveragePooling1D()(x)

        return Model(input, x)

    def lstm_encoder_network_3_gru(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(GRU(unit_number_of_lstm, return_sequences=True, dropout = lstm_dropout_rate,
                              recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(input)                    
        x = AveragePooling1D(pool_size=2, strides=None)(x)
        # x = Bidirectional(GRU(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x)      
        # x = Bidirectional(GRU(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
        #                       recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
        # x = AveragePooling1D(pool_size=2, strides=None)(x)                      
        x = tf.transpose(x, perm=[0,2,1])
        x = attention(return_sequences=True)(x)
        x = tf.transpose(x, perm=[0,2,1])        
        x = GlobalAveragePooling1D()(x)

        return Model(input, x)


    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(512)(input)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(0.25)(x)
      x = Dense(64)(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(0.15)(x)
      x = Dense(11, activation="softmax")(x)
      return Model(input, x)

    input_shape = range_doppler_concat_shuffle.shape[1:3]
    base_network_lstm_1 = lstm_encoder_network_1(input_shape)
    lstm_input_1  = Input(shape=input_shape)
    processed_lstm_1  = base_network_lstm_1(lstm_input_1)

    input_shape = spectrogram_concat_shuffle.shape[1:3]
    base_network_lstm_2 = lstm_encoder_network_2(input_shape)
    lstm_input_2  = Input(shape=input_shape)
    processed_lstm_2  = base_network_lstm_1(lstm_input_2)

    input_shape = third_spectrogram_concat_shuffle.shape[1:3]
    base_network_lstm_3 = lstm_encoder_network_3(input_shape)
    lstm_input_3  = Input(shape=input_shape)
    processed_lstm_3  = base_network_lstm_1(lstm_input_3)


    base_network_lstm_1_gru = lstm_encoder_network_1_gru(input_shape)
    processed_lstm_1_gru  = base_network_lstm_1_gru(lstm_input_1)

    base_network_lstm_2_gru = lstm_encoder_network_2_gru(input_shape)
    processed_lstm_2_gru  = base_network_lstm_1_gru(lstm_input_2)

    base_network_lstm_3_gru = lstm_encoder_network_3_gru(input_shape)
    processed_lstm_3_gru  = base_network_lstm_1_gru(lstm_input_3)    

    concat_layer = Concatenate()([processed_lstm_1, processed_lstm_2, processed_lstm_3, processed_lstm_1_gru, processed_lstm_2_gru, processed_lstm_3_gru])

    base_decoder_network = decoder_for_concat((concat_layer.shape[1]))
    out = base_decoder_network(concat_layer)

    model = Model(inputs=[lstm_input_1,lstm_input_2,lstm_input_3], outputs=[out])
    if repeat_run_number == 0:
      print(model.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=25, verbose=0,restore_best_weights=True, mode='min')
    t = time.time()
    history = model.fit((range_doppler_augmented_image,spectrogram_augmented_image,third_spectrogram_augmented_image),(spectrogram_concat_label_shuffle_concat),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    verbose = 0,
                    callbacks=[earlyStopping],
                    validation_data = ((validation_range_doppler,validation_spectrogram,validation_third_spectrogram) , (spectrogram_validation_labels)))
    elapsed = time.time() - t
    tf.keras.models.load_model
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test,spectrogram_concat_shuffle_test,third_spectrogram_concat_shuffle_test],\
                                               [spectrogram_concat_label_shuffle_test],
                  batch_size=batch_size)
    test_accuracy_per_fold.append(test_accuracy)
    print(test_accuracy_per_fold)

  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  print(test_accuracy_per_run)

print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')

print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 170, 128)]   0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 170, 128)]   0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 170, 128)]   0           []                               
                                                                                                  
 model (Functional)             (None, 128)          99029       ['input_2[0][0]',                
                                                                  'input_4[0][0]',          

In [16]:
t_test = time.time()
y_test_predicted = model.predict((range_doppler_concat_shuffle_test[0:1,:,:,:],spectrogram_concat_shuffle_test[0:1,:,:,:],third_spectrogram_concat_shuffle_test[0:1,:,:,:]))
test_elapsed = time.time() - t_test 
print(f'Time elapsed during test time: {"{:.2f}".format(test_elapsed)}, sec')

Time elapsed during test time: 4.62, sec


In [17]:
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)} sec')
print(f'Time elapsed during test time: {"{:.3f}".format(test_elapsed)} sec')

Mean test accuracy is 0.932, max test accuracy is 0.941, min test accuracy is 0.916,  std of test accuracy is 0.009.
Time elapsed through all process: 268.079 sec
Time elapsed during test time: 4.622 sec
